In [3]:
import librosa
import IPython.display as ipd
import nemo
import nemo.collections.asr as nemo_asr
import pytorch_lightning as pl
from ruamel.yaml import YAML
from omegaconf import DictConfig
from nemo.core.classes import ModelPT
import os

In [4]:
yaml = YAML(typ='safe')
params_path = r"C:\Users\vdovichev\Documents\VKR\models_and_configs\augmentationConf.yaml"
with open(params_path, encoding="utf-8") as f:
    params = yaml.load(f)

In [6]:
asr_model = ModelPT.restore_from(r"C:\Users\vdovichev\Documents\VKR\models_and_configs\bestQuartz1206.nemo")
# asr_model = nemo_asr.models.EncDecCTCModel.load_from_checkpoint(r"C:\Users\vdovichev\Documents\FineTune\models_configs\wer=0.3511.ckpt")

[NeMo W 2024-06-12 16:41:06 modelPT:161] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath:
    - /home/jupyter/datasphere/datasets/RuDevices/RuDevicesDataset/manifestRuDevices.jsonl
    - /home/jupyter/datasphere/datasets/STTandCustomer/OpenSTTruDatasets/asr_calls_2_val/manifestSTTcalls.jsonl
    - /home/jupyter/datasphere/datasets/STTandCustomer/OpenSTTruDatasets/buriy_audiobooks_2_val/manifestSTTaudiobooks.jsonl
    - /home/jupyter/datasphere/datasets/STTandCustomer/OpenSTTruDatasets/public_youtube700_val/manifestSTTyoutube.jsonl
    sample_rate: 16000
    labels:
    - ' '
    - а
    - б
    - в
    - г
    - д
    - е
    - ж
    - з
    - и
    - й
    - к
    - л
    - м
    - н
    - о
    - п
    - р
    - с
    - т
    - у
    - ф
    - х
    - ц
    - ч
    - ш
    - щ
    - ъ
    - ы
    - ь
    - э
    - ю
    - я

[NeMo I 2024-06-12 16:41:06 features:289] PADDING: 16
[NeMo I 2024-06-12 16:41:07 save_restore_connector:249] Model EncDecCTCModel was successfully restored from C:\Users\vdovichev\Documents\VKR\models_and_configs\bestQuartz1206.nemo.


In [9]:
asr_model.setup_optimization(optim_config= DictConfig(params['optim']))
asr_model.setup_validation_data(val_data_config=params['validation_ds'])
asr_model.setup_training_data(train_data_config=params['train_ds'])

[NeMo W 2024-06-12 16:42:48 modelPT:617] Trainer wasn't specified in model constructor. Make sure that you really wanted it.


[NeMo I 2024-06-12 16:42:48 modelPT:728] Optimizer config = Novograd (
    Parameter Group 0
        amsgrad: False
        betas: [0.9, 0.98]
        eps: 1e-08
        grad_averaging: False
        lr: 0.005
        weight_decay: 0.001
    )


[NeMo W 2024-06-12 16:42:48 lr_scheduler:890] Neither `max_steps` nor `iters_per_batch` were provided to `optim.sched`, cannot compute effective `max_steps` !
    Scheduler will not be instantiated !


[NeMo I 2024-06-12 16:42:49 collections:196] Dataset loaded with 2050 files totalling 1.83 hours
[NeMo I 2024-06-12 16:42:49 collections:197] 0 files were filtered totalling 0.00 hours
[NeMo I 2024-06-12 16:42:49 collections:196] Dataset loaded with 1100 files totalling 0.33 hours
[NeMo I 2024-06-12 16:42:49 collections:197] 0 files were filtered totalling 0.00 hours
[NeMo I 2024-06-12 16:42:49 collections:196] Dataset loaded with 843 files totalling 2.33 hours
[NeMo I 2024-06-12 16:42:49 collections:197] 0 files were filtered totalling 0.00 hours
[NeMo I 2024-06-12 16:42:58 collections:196] Dataset loaded with 119304 files totalling 109.23 hours
[NeMo I 2024-06-12 16:42:58 collections:197] 1 files were filtered totalling 0.01 hours


In [11]:
print(asr_model._cfg)

{'sample_rate': 16000, 'repeat': 5, 'dropout': 0.0, 'separable': True, 'labels': [' ', 'а', 'б', 'в', 'г', 'д', 'е', 'ж', 'з', 'и', 'й', 'к', 'л', 'м', 'н', 'о', 'п', 'р', 'с', 'т', 'у', 'ф', 'х', 'ц', 'ч', 'ш', 'щ', 'ъ', 'ы', 'ь', 'э', 'ю', 'я'], 'optim': {'name': 'novograd', 'lr': 0.005, 'betas': [0.9, 0.98], 'weight_decay': 0.001, 'sched': {'name': 'CosineAnnealing', 'warmup_steps': 10000, 'warmup_ratio': None, 'min_lr': 0.0005, 'last_epoch': -1}}, 'train_ds': {'manifest_filepath': ['C:\\Users\\vdovichev\\Documents\\proj1-STT\\DatasetForFineTune\\RuDevicesDataset\\manifestRuDevices.jsonl', 'C:\\Users\\vdovichev\\Documents\\proj1-STT\\DatasetForFineTune\\OpenSTTruDatasets\\asr_calls_2_val\\manifestSTTcalls.jsonl', 'C:\\Users\\vdovichev\\Documents\\proj1-STT\\DatasetForFineTune\\OpenSTTruDatasets\\buriy_audiobooks_2_val\\manifestSTTaudiobooks.jsonl', 'C:\\Users\\vdovichev\\Documents\\proj1-STT\\DatasetForFineTune\\OpenSTTruDatasets\\public_youtube700_val\\manifestSTTyoutube.jsonl'], '

In [12]:
import torch

if torch.cuda.is_available():
  accelerator = 'gpu'
else:
  accelerator = 'gpu'

EPOCHS = 20  # 100 epochs would provide better results

trainer = pl.Trainer(devices=1,
                      accelerator=accelerator,
                      max_epochs=EPOCHS,
                      enable_checkpointing=False,
                      logger=False,
                      limit_val_batches = 1.0,
                      check_val_every_n_epoch = 1,
                      val_check_interval = 1.0)

# Setup model with the trainer
asr_model.set_trainer(trainer)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
`Trainer(limit_val_batches=1.0)` was configured so 100% of the batches will be used..
`Trainer(val_check_interval=1.0)` was configured so validation will run at the end of the training epoch..


In [13]:
from nemo.utils import exp_manager
from omegaconf import OmegaConf

# Environment variable generally used for multi-node multi-gpu training.
# In notebook environments, this flag is unnecessary and can cause logs of multiple training runs to overwrite each other.
os.environ.pop('NEMO_EXPM_VERSION', None)

config = exp_manager.ExpManagerConfig(
    exp_dir="C:\\Users\\vdovichev\\Documents\\proj1-STT\\FineTune\\nemo_experiments\\",
    name=f"test2",
    create_tensorboard_logger = True,
    create_checkpoint_callback = True,
    checkpoint_callback_params=exp_manager.CallbackParams(
        monitor="val_wer",
        mode="min",
        always_save_nemo=True,
        save_best_model=True,
    ),
    create_wandb_logger = False,
    wandb_logger_kwargs={
        "name": None,
        "project": None,
    },
)

config = OmegaConf.structured(config)

logdir = exp_manager.exp_manager(trainer, config)

[NeMo I 2024-06-12 16:46:16 exp_manager:386] Experiments will be logged at C:\Users\vdovichev\Documents\proj1-STT\FineTune\nemo_experiments\test2\2024-06-12_16-46-16
[NeMo I 2024-06-12 16:46:16 exp_manager:825] TensorboardLogger has been set up


In [11]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [ ]:
%tensorboard --logdir "C:\\Users\\vdovichev\\Documents\\proj1-STT\\FineTune\\nemo_experiments\\test2\\" --port 8888

In [15]:
trainer.validate(asr_model)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Validation DataLoader 0: 100%|██████████| 684/684 [00:24<00:00, 27.83it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃      Validate metric      ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│        global_step        │           18.0            │
│         val_loss          │    12.039067268371582     │
│          val_wer          │    0.25759127736091614    │
└───────────────────────────┴───────────────────────────┘

[{'global_step': 18.0,
  'val_loss': 12.039067268371582,
  'val_wer': 0.25759127736091614}]

In [14]:
trainer.fit(asr_model)

You are using a CUDA device ('NVIDIA GeForce RTX 4070 Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


[NeMo I 2024-06-12 16:46:53 modelPT:728] Optimizer config = Novograd (
    Parameter Group 0
        amsgrad: False
        betas: [0.9, 0.98]
        eps: 1e-08
        grad_averaging: False
        lr: 0.005
        weight_decay: 0.001
    )
[NeMo I 2024-06-12 16:46:53 lr_scheduler:910] Scheduler "<nemo.core.optim.lr_scheduler.CosineAnnealing object at 0x0000027491663C10>" 
    will be used during training (effective maximum steps = 596520) - 
    Parameters : 
    (warmup_steps: 10000
    warmup_ratio: null
    min_lr: 0.0005
    last_epoch: -1
    max_steps: 596520
    )



  | Name              | Type                              | Params
------------------------------------------------------------------------
0 | preprocessor      | AudioToMelSpectrogramPreprocessor | 0     
1 | encoder           | ConvASREncoder                    | 18.9 M
2 | decoder           | ConvASRDecoder                    | 34.9 K
3 | loss              | CTCLoss                           | 0     
4 | spec_augmentation | SpectrogramAugmentation           | 0     
5 | _wer              | WER                               | 0     
------------------------------------------------------------------------
18.9 M    Trainable params
0         Non-trainable params
18.9 M    Total params
75.718    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

[NeMo W 2024-06-12 16:46:53 nemo_logging:349] c:\Users\vdovichev\anaconda3\envs\nemo\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:438: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 20 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
      rank_zero_warn(
    


[NeMo W 2024-06-12 16:46:54 nemo_logging:349] c:\Users\vdovichev\anaconda3\envs\nemo\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:438: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 20 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
      rank_zero_warn(
    


Training: 0it [00:00, ?it/s][NeMo I 2024-06-12 16:46:54 preemption:56] Preemption requires torch distributed to be initialized, disabling preemption
Epoch 0:   0%|          | 18/7457 [00:10<1:12:10,  1.72it/s, v_num=6-16, train_step_timing in s=0.267]

[NeMo W 2024-06-12 16:47:05 nemo_logging:349] c:\Users\vdovichev\anaconda3\envs\nemo\lib\site-packages\pytorch_lightning\trainer\call.py:53: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
      rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")
    
